In [1]:
import time

import mkl_random

pessoas = [('Lisbon', 'LIS'), ('Madrid', 'MAD'), ('Paris', 'CDG'), ('Dublin', 'DUB'), ('Brussels', 'BRU'), ('Lodon', 'LHR')]

In [2]:
destino = 'FCO'

In [3]:
voos = {('BRUNA','FCO'): ['15:44','18:55', 382]}

In [4]:
voos = {}
for linha in open('flights.txt'):
    origin, destino, partida, chegada, preco = linha.split(',')
    voos.setdefault((origin, destino), []).append([partida, chegada, int(preco.strip())])

print(voos)

{('FCO', 'LIS'): [['6:19', '8:13', 239], ['8:04', '10:59', 136], ['9:31', '11:43', 210], ['11:07', '13:24', 171], ['12:31', '14:02', 234], ['14:05', '15:47', 226], ['15:07', '17:21', 129], ['16:35', '18:56', 144], ['18:25', '20:34', 205], ['20:05', '21:44', 172]], ('LIS', 'FCO'): [['6:11', '8:31', 249], ['7:39', '10:24', 219], ['9:15', '12:03', 99], ['11:08', '13:07', 175], ['12:18', '14:56', 172], ['13:37', '15:08', 250], ['15:03', '16:42', 135], ['16:51', '19:09', 147], ['18:12', '20:17', 242], ['20:05', '22:06', 261]], ('FCO', 'MAD'): [['6:03', '8:43', 219], ['7:50', '10:08', 164], ['9:11', '10:42', 172], ['10:33', '13:11', 132], ['12:08', '14:47', 231], ['14:19', '17:09', 190], ['15:04', '17:23', 189], ['17:06', '20:00', 95], ['18:33', '20:22', 143], ['19:32', '21:25', 160]], ('MAD', 'FCO'): [['6:05', '8:32', 174], ['8:25', '10:34', 157], ['9:42', '11:32', 169], ['11:01', '12:39', 260], ['12:44', '14:17', 134], ['14:22', '16:32', 126], ['15:58', '18:40', 173], ['16:43', '19:00', 24

In [5]:
def imprime_calendario(calendario):
    voo_id = -1
    preco_total = 0
    for i in range(len(calendario) // 2):
        nome = pessoas[i][0]
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        preco_total += voo_ida[2]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]
        preco_total += voo_volta[2]
        print('%10s%10s %5s-%5s U$%3s %5s-%5s U$%3s' % (nome, origem, voo_ida[0], voo_ida[1], voo_ida[2],
                                                        voo_volta[0], voo_volta[1], voo_volta[2]))
    print('Preço total: ', preco_total)

In [6]:
imprime_calendario([1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3])

    Lisbon       LIS  7:39-10:24 U$219 12:31-14:02 U$234
    Madrid       MAD 11:01-12:39 U$260  9:11-10:42 U$172
     Paris       CDG 17:07-20:04 U$291 11:08-14:38 U$262
    Dublin       DUB 15:27-17:18 U$151 10:33-12:03 U$ 74
  Brussels       BRU  9:08-12:12 U$364 12:20-16:34 U$500
     Lodon       LHR 13:40-15:38 U$137 10:32-13:16 U$139
Preço total:  2803


In [7]:
def get_minutos(hora):
    t = time.strptime(hora, '%H:%M')
    minutos = t[3] * 60 + t[4]
    return minutos

In [8]:
get_minutos('7:39'), get_minutos('10:32')

(459, 632)

# Função de avaliação (Fitness function)

In [9]:
def funcao_avaliacao(calendario):
    preco_total = 0
    ultima_chegada = 0
    primeira_partida = 1439

    voo_id = -1
    for i in range(len(calendario) // 2):
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        preco_total += voo_ida[2]
        preco_total += voo_volta[2]

        if ultima_chegada < get_minutos(voo_ida[1]):
            ultima_chegada = get_minutos(voo_ida[1])
        if primeira_partida > get_minutos(voo_volta[0]):
            primeira_partida = get_minutos(voo_volta[0])

    espera_total = 0
    voo_id = -1

    for i in range(len(calendario) // 2):
        origem = pessoas[i][1]
        voo_id += 1
        voo_ida = voos[(origem, destino)][calendario[voo_id]]
        voo_id += 1
        voo_volta = voos[(destino, origem)][calendario[voo_id]]

        espera_total += ultima_chegada - get_minutos(voo_ida[1])
        espera_total += get_minutos(voo_volta[0]) - primeira_partida

    return espera_total + preco_total


In [10]:
funcao_avaliacao([1, 4, 3, 1, 7, 3, 6, 3, 2, 4, 5, 3])

5798

In [11]:
dominio = [(0,9)] * (len(pessoas) * 2)
dominio

[(0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9),
 (0, 9)]

In [40]:
import random


def mutacao(dominio, passo, calendario, probabilidade):
    gene = random.randint(0, len(dominio) - 1)
    mutante = calendario
    if random.random() < probabilidade:
        if calendario[gene] != dominio[gene][0]:
            mutante = calendario[0:gene] + [calendario[gene] - passo] + calendario[gene + 1:]
        elif calendario[gene] != dominio[gene][1]:
            mutante = calendario[0:gene] + [calendario[gene] + passo] + calendario[gene + 1:]
    return mutante

In [13]:
mutacao(dominio, 1, [6, 7, 6, 7, 3, 9, 7, 7, 0, 7, 6 ,7])

6


[6, 7, 6, 7, 3, 9, 6, 7, 0, 7, 6, 7]

# Crossover

In [14]:
def crossover(dominio, individuo1, individuo2):
    gene = random.randint(1, len(dominio) - 2)
    return individuo1[0:gene] + individuo2[gene:]

In [15]:
s1 = [1,4,3,2,7,3,6,3,2,4,5,3]
s2 = [0,1,2,5,8,9,2,3,5,1,0,6]
crossover(dominio, s1, s2)

[1, 1, 2, 5, 8, 9, 2, 3, 5, 1, 0, 6]

In [35]:
def algoritmo_genetico(dominio, funcao_avaliacao, tamanho_populacao = 100, passo = 1, elitismo = 0.2, numero_geracoes = 500, probabilidade_mutacao = 0.05):
    populacao = []
    for i in range(tamanho_populacao):
        individuo = [random.randint(dominio[i][0], dominio[i][1]) for i in range(len(dominio))]
        populacao.append(individuo)
    numero_elitismo = int(elitismo * tamanho_populacao)
    custos = []
    for i in range(numero_geracoes):
        custos = [(funcao_avaliacao(individuo), individuo) for individuo in populacao]
        custos.sort()
        individuos_ordenados = [individuo for (custo, individuo) in custos]
        populacao = individuos_ordenados[0:numero_elitismo]

        while len(populacao) < tamanho_populacao:
            i1 = random.randint(0, numero_elitismo)
            i2 = random.randint(0, numero_elitismo)

            novo_individuo = crossover(dominio, individuos_ordenados[i1], individuos_ordenados[i2])

            mutacao_novos_individuos = mutacao(dominio, passo, novo_individuo, probabilidade_mutacao)
            populacao.append(mutacao_novos_individuos)
    return custos[0][1]

In [52]:
solucao = algoritmo_genetico(dominio, funcao_avaliacao, numero_geracoes = 100, tamanho_populacao=100,
                             elitismo = 0.2, probabilidade_mutacao = 0.05)

In [46]:
solucao

[4, 2, 4, 2, 3, 4, 5, 3, 3, 2, 5, 3]

In [53]:
funcao_avaliacao(solucao)

2764

In [54]:
imprime_calendario(solucao)

    Lisbon       LIS 15:03-16:42 U$135  8:04-10:59 U$136
    Madrid       MAD 15:58-18:40 U$173  7:50-10:08 U$164
     Paris       CDG 15:34-18:11 U$326  8:23-11:07 U$143
    Dublin       DUB 17:11-18:30 U$108 10:33-12:03 U$ 74
  Brussels       BRU 13:54-18:02 U$294  9:49-13:51 U$229
     Lodon       LHR 15:23-17:25 U$232  8:19-11:16 U$122
Preço total:  2136


In [56]:
menor_solucao = [5000,[]]
for i in range(0,100):
    sol = algoritmo_genetico(dominio, funcao_avaliacao, numero_geracoes = 100, tamanho_populacao=100,
                             elitismo = 0.2, probabilidade_mutacao = 0.05)
    if funcao_avaliacao(sol) < menor_solucao[0]:
        menor_solucao[0] = funcao_avaliacao(sol)
        menor_solucao[1] = sol

print(menor_solucao[0])
imprime_calendario(menor_solucao[1])

2354
    Lisbon       LIS 12:18-14:56 U$172  8:04-10:59 U$136
    Madrid       MAD 12:44-14:17 U$134  7:50-10:08 U$164
     Paris       CDG 11:28-14:40 U$248  8:23-11:07 U$143
    Dublin       DUB 12:34-15:02 U$109  8:23-10:28 U$149
  Brussels       BRU 10:30-14:57 U$290  9:49-13:51 U$229
     Lodon       LHR 12:08-14:59 U$149  8:19-11:16 U$122
Preço total:  2045
